# 🧠 Train-Your-Own Emotion Detection Model (No Pretrained Models)
This notebook defines, trains, and uses your own CNN model from scratch.

In [ ]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [ ]:

# === CONFIGURATION ===
image_size = (96, 96)
batch_size = 32
epochs = 25
train_dir = "dataset/train"
val_dir = "dataset/val"
test_dir = "dataset/test"
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)


In [ ]:

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(train_dir, target_size=image_size, color_mode="grayscale", batch_size=batch_size, class_mode="categorical")
val_data = val_gen.flow_from_directory(val_dir, target_size=image_size, color_mode="grayscale", batch_size=batch_size, class_mode="categorical")
test_data = test_gen.flow_from_directory(test_dir, target_size=image_size, color_mode="grayscale", batch_size=batch_size, class_mode="categorical")

class_labels = list(train_data.class_indices.keys())
num_classes = len(class_labels)


In [ ]:

def build_custom_emotion_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    return Model(inputs, outputs)

model = build_custom_emotion_model((96, 96, 1), num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:

# === TRAIN MODEL ===
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(os.path.join(model_dir, 'best_weights.h5'), save_best_only=True, monitor='val_accuracy', mode='max')

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


In [ ]:

# === SAVE MODEL ===
model_json = model.to_json()
with open(os.path.join(model_dir, "emotion_model.json"), "w") as json_file:
    json_file.write(model_json)
model.save_weights(os.path.join(model_dir, "emotion_model_weights.h5"))
model.save("saved_model")
print("✅ Model saved (JSON, weights, SavedModel)")


In [ ]:

# === EVALUATE MODEL ===
test_data.reset()
Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_data.classes

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))


In [ ]:

# === INFER IMAGES LOCALLY ===
def predict_emotions_from_images(folder):
    for img_file in os.listdir(folder):
        if not img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            continue

        path = os.path.join(folder, img_file)
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, image_size)

        face_img = resized.astype("float32") / 255.0
        face_img = np.expand_dims(face_img.reshape(image_size[0], image_size[1], 1), axis=0)

        preds = model.predict(face_img)[0]
        label = class_labels[np.argmax(preds)]
        confidence = np.max(preds)

        print(f"{img_file} → {label} ({confidence*100:.2f}%)")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"{label} ({confidence*100:.2f}%)")
        plt.axis('off')
        plt.show()

# Uncomment to test:
# predict_emotions_from_images("images")
